# Simulating $Q(w,a)$ values, verifying equations and such

In [ ]:
# Packages

import math

import numpy as np
import pandas as pd

from scipy.optimize import fsolve, minimize, Bounds, LinearConstraint
from scipy.integrate import quad
from scipy import linspace, meshgrid, arange, empty, concatenate, newaxis, shape

import matplotlib.pyplot as plt

from collections import deque

import time

from statistics import mean 

In [ ]:
###########################################################################################################################################################################################
# Simulation variables

# True arrival rate
global lam

# Service rate
global service_par

# True joining probability parameters
global theta_1, theta_2

# Number of servers
global s

# Admission price - what we control
global control_price

# Total customers that we shall generate
global n

############################################################################################################################################################################################

def generate_arrival_process(x): # x is the arrival rate
    global interarrival_times
    global arrival_times
    
    interarrival_times = np.random.exponential(1/x, n)
    arrival_times = np.cumsum(interarrival_times)
    arrival_times -= arrival_times[0]
    
###########################################################################################################################################################################################
    
def generate_service_times(x):
    global service_times
    global mean_service_time
    
    # Gamma distribution - mean = x[0]x[1] and variance = x[0]x[1]^2
    # mean_service_time = x[0]*x[1]
    # service_times = np.random.gamma(x[0], x[1], n)
    
    # Exponential distribution
    mean_service_time = 1/x
    service_times = np.random.exponential(1/x, n)
    
###########################################################################################################################################################################################

class Customer(): # Creating an object "Customer" with relevant information
    # Price paid by customer
    price_paid = 0
    
    # Time of arrival
    arrival_time = 0
    
    # Time of departure
    departure_time = 0
    
    # Time taken for servicing
    service_time = 0
    
    # Time spent waiting in queue (does not include service time)
    waiting_time = 0
    
    # Server number which served the customer
    server = 0
    
    # Does the customer join? 0 if yes, 1 if no
    joining_decision = 0
    
    # Customer number (basically assigned between 0 and n-1)
    number = 0
    
    # Observed number of customers in system upon arrival and before joining
    observed_system_occupancy = 0
    
    # Jump caused in virtual waiting time
    jump_virtual_waiting_time = 0
    
###########################################################################################################################################################################################

In [ ]:
def estimator_gradient_revenue(w, a, p, par_vec):
    if a == 1:
        est = gradient_joining_probability(w, p) * approx_Q(w, 1, p, par_vec)/joining_probability(w, p)
    elif a == 0:
        est = -gradient_joining_probability(w, p) * approx_Q(w, 0, p, par_vec)/(1 - joining_probability(w, p))
    
    # est = gradient_joining_probability(w, p) * (approx_Q(w, 1, p, par_vec)/joining_probability(w, p) - approx_Q(w, 0, p, par_vec)/(1-joining_probability(w, p)))
    return est

def approx_Q(w, a, p, par_vec):
    G = par_vec[0] * gradient_joining_probability(w, p) * (2*a-1)/((2*a-1)*joining_probability(w, p) + (1-a))
    return G
    
def gradient_approx_Q(w, a, p):
    grad = gradient_joining_probability(w, p) * (2*a-1)/((2*a-1)*joining_probability(w, p) + (1-a))
    return grad
    
def joining_probability(w, p):
    prob = np.exp(-theta_1*p - theta_2*w)
    return prob
    
def gradient_joining_probability(w, p):
    grad = np.exp(-theta_1*p - theta_2*w)*(-theta_1)
    return grad

In [ ]:
# Obtain Q-value for one sample path given fixed price p, R(p), initial state and initial action
# Basically, whatever is inside the expectation, for a fixed realization.

# Q(s,a) = \sum_{i=1}^{inf} E[r_i - R(p) | s_0 = s, a_0 = a] where r_i = lambda p a_i

def queue_simulation_1(initial_workload, initial_action, p, theoretical_revenue):
    
# Idea

# Create array of workload of servers - Workload[s]
# Calculate min workload and server with min workload. If min workload > incoming customer patience, then customer balks. 
# Else, new workload (of min workload server) = current workload + service time of incoming customer
# This array will be updated during every arrival (only based on updated values we can decide balking or not!)
# If customer is non-balking, it is easy to get their waiting time in queue (= min workload amongst all servers)

# Maintain an array of "s" queues, each queue i literally represents queue in front of server i - append and pop customers during each arrival event
# Maintain a list to keep track of residual service times
    
    # Simulation variables
    global customers
    global server_workloads
    global dynamic_queues
    global dynamic_residual_service_times
    
    # Post processing variables
    global effective_arrival_times
    global effective_interarrival_times
    global departure_times
    global waiting_times
    global upward_jump_virtual_waiting_times
    
    global long_run_average_cost_iterates
    global long_run_average_revenue_iterates
    
    # global sampled_states
    
##################################################################################################################################    
    # Initialization
    customers = [Customer() for i in range(n)]
    server_workloads = initial_workload
    
    # New variables to keep track of
    dynamic_residual_service_times = initial_workload # Keeps track of residual service times just after arrival events during the simulation
    dynamic_queues = deque()

    # Relevant information to calculate and store
    effective_arrival_times = []
    effective_interarrival_times = []
    departure_times = []
    waiting_times = []
    upward_jump_virtual_waiting_times = []
    
    # Storing relevant information
    long_run_average_revenue = 0
    long_run_average_revenue_iterates = [long_run_average_revenue]
    
    # Sampling points from a Poisson process
    # sampled_states = []
    # sampling_interarrival_time = arrival_times[-1]/100
    # next_sampling_time = sampling_interarrival_time
    
##################################################################################################################################   
    
    Q_value = 0
    price = p
    
    i = 0
    for cust in customers:
        
        if i >= 10000 and arrival_times[i] >= next_sampling_time:
            sampled_states.append(server_workloads)
            next_sampling_time += sampling_interarrival_time
        
        
        # Storing customer information
        cust.number = i
        cust.arrival_time = arrival_times[i]
        cust.service_time = service_times[i]
        
        # Calculates number in system observed by incoming arrival
        # system_size_at_joining_instant = len(dynamic_queues)
        # cust.observed_system_occupancy = system_size_at_joining_instant
        
        # Joining decision
        if i == 0:
            cust.waiting_time = server_workloads
            cust.joining_decision = initial_action
        else:
            cust.waiting_time = server_workloads
            joining_probability = np.exp(-theta_1*price -theta_2*cust.waiting_time)
            cust.joining_decision = np.random.binomial(1, joining_probability)
        
        Q_value += (lam*price*cust.joining_decision - theoretical_revenue)
        
        
        if cust.joining_decision == 1:
            dynamic_queues.append(i) # Customer joins queue corresponding to minimum workload server
            if len(dynamic_queues) == 1:
                dynamic_residual_service_times = cust.service_time

            effective_arrival_times.append(cust.arrival_time)

            cust.server = 1
            cust.departure_time = cust.arrival_time + cust.waiting_time + cust.service_time
            
            # Jump in virtual waiting time calculation
            server_workloads += cust.service_time # Adds arriving customer's service time to minimum workload server
            cust.jump_virtual_waiting_time = cust.service_time # Adding new virtual waiting time AFTER customer arrival

            waiting_times.append(cust.waiting_time)
            departure_times.append(cust.departure_time)
            upward_jump_virtual_waiting_times.append(cust.jump_virtual_waiting_time)
        
            
        if i != n-1:
            # Time to update residual service times here. Mathematical formulation of the code is as follows:-
            # Suppose service times of a queue corresponding to a server are: 5(residual service time of customer being served),20,22,17,23,28,... and time since last arrival is 50,
            # Then, new queue becomes 14,23,28,... 
            for j in range(1):
                if len(dynamic_queues) == 0:
                    dynamic_residual_service_times = 0
                    continue
                
                time_until_next_arrival = arrival_times[i+1] - arrival_times[i]

                if time_until_next_arrival > dynamic_residual_service_times:
                    time_until_next_arrival -= dynamic_residual_service_times
                    dynamic_residual_service_times = 0
                    dynamic_queues.popleft()
                else:
                    dynamic_residual_service_times -= time_until_next_arrival
                    continue

                while len(dynamic_queues)!=0 and time_until_next_arrival > customers[dynamic_queues[0]].service_time:
                    time_until_next_arrival -= customers[dynamic_queues[0]].service_time
                    dynamic_queues.popleft()

                if len(dynamic_queues) == 0:
                    dynamic_residual_service_times = 0
                    continue
                else:
                    dynamic_residual_service_times = customers[dynamic_queues[0]].service_time - time_until_next_arrival
            
            
            server_workloads -= (arrival_times[i+1] - arrival_times[i]) # Subtracting time passed until next arrival from all server workloads
            server_workloads = max(server_workloads, 0)  # All those with workload < 0 are converted to 0
        
        i += 1

##################################################################################################################################
        
    # Post processing
    
#     effective_arrival_times = np.array(effective_arrival_times)
#     effective_interarrival_times = [effective_arrival_times[0]] + [j-i for i,j in zip(effective_arrival_times, effective_arrival_times[1:])]
#     effective_interarrival_times = np.array(effective_interarrival_times)
#     departure_times.sort()
#     departure_times = np.array(departure_times)
#     waiting_times = np.array(waiting_times)
#     upward_jump_virtual_waiting_times = np.array(upward_jump_virtual_waiting_times)
    
#     all_waiting_times = np.array(all_waiting_times)

    return Q_value

In [ ]:
# Obtain good estimates for R(p) given price p 

def queue_simulation_2(p):
    
# Idea

# Create array of workload of servers - Workload[s]
# Calculate min workload and server with min workload. If min workload > incoming customer patience, then customer balks. 
# Else, new workload (of min workload server) = current workload + service time of incoming customer
# This array will be updated during every arrival (only based on updated values we can decide balking or not!)
# If customer is non-balking, it is easy to get their waiting time in queue (= min workload amongst all servers)

# Maintain an array of "s" queues, each queue i literally represents queue in front of server i - append and pop customers during each arrival event
# Maintain a list to keep track of residual service times
    
    # Simulation variables
    global customers
    global server_workloads
    global dynamic_queues
    global dynamic_residual_service_times
    
    # Post processing variables
    global effective_arrival_times
    global effective_interarrival_times
    global departure_times
    global waiting_times
    global upward_jump_virtual_waiting_times
    
    global long_run_average_cost_iterates
    global long_run_average_revenue_iterates
    
##################################################################################################################################    
    # Initialization
    customers = [Customer() for i in range(n)]
    server_workloads = 0
    
    # New variables to keep track of
    dynamic_residual_service_times = 0 # Keeps track of residual service times just after arrival events during the simulation
    dynamic_queues = deque()

    # Relevant information to calculate and store
    effective_arrival_times = []
    effective_interarrival_times = []
    departure_times = []
    waiting_times = []
    upward_jump_virtual_waiting_times = []
    
    # Storing relevant information
    long_run_average_revenue = 0
    long_run_average_revenue_iterates = [long_run_average_revenue]
    
##################################################################################################################################   
    
    total_revenue = 0
    average_revenue = 0
    
    price = p
    
    i = 0
    for cust in customers:
        
        # Storing customer information
        cust.number = i
        cust.arrival_time = arrival_times[i]
        cust.service_time = service_times[i]
        
        # Calculates number in system observed by incoming arrival
        # system_size_at_joining_instant = len(dynamic_queues)
        # cust.observed_system_occupancy = system_size_at_joining_instant
        
        # Joining decision
        cust.waiting_time = server_workloads
        joining_probability = np.exp(-theta_1*price -theta_2*cust.waiting_time)
        cust.joining_decision = np.random.binomial(1, joining_probability)

        
        if cust.joining_decision == 1:
            total_revenue += price
            
            dynamic_queues.append(i) # Customer joins queue corresponding to minimum workload server
            if len(dynamic_queues) == 1:
                dynamic_residual_service_times = cust.service_time

            effective_arrival_times.append(cust.arrival_time)

            cust.server = 1
            cust.departure_time = cust.arrival_time + cust.waiting_time + cust.service_time
            
            # Jump in virtual waiting time calculation
            server_workloads += cust.service_time # Adds arriving customer's service time to minimum workload server
            cust.jump_virtual_waiting_time = cust.service_time # Adding new virtual waiting time AFTER customer arrival

            waiting_times.append(cust.waiting_time)
            departure_times.append(cust.departure_time)
            upward_jump_virtual_waiting_times.append(cust.jump_virtual_waiting_time)
        
            
        if i != n-1:
            # Time to update residual service times here. Mathematical formulation of the code is as follows:-
            # Suppose service times of a queue corresponding to a server are: 5(residual service time of customer being served),20,22,17,23,28,... and time since last arrival is 50,
            # Then, new queue becomes 14,23,28,... 
            for j in range(1):
                if len(dynamic_queues) == 0:
                    dynamic_residual_service_times = 0
                    continue
                
                time_until_next_arrival = arrival_times[i+1] - arrival_times[i]

                if time_until_next_arrival > dynamic_residual_service_times:
                    time_until_next_arrival -= dynamic_residual_service_times
                    dynamic_residual_service_times = 0
                    dynamic_queues.popleft()
                else:
                    dynamic_residual_service_times -= time_until_next_arrival
                    continue

                while len(dynamic_queues)!=0 and time_until_next_arrival > customers[dynamic_queues[0]].service_time:
                    time_until_next_arrival -= customers[dynamic_queues[0]].service_time
                    dynamic_queues.popleft()

                if len(dynamic_queues) == 0:
                    dynamic_residual_service_times = 0
                    continue
                else:
                    dynamic_residual_service_times = customers[dynamic_queues[0]].service_time - time_until_next_arrival
            
            
            server_workloads -= (arrival_times[i+1] - arrival_times[i]) # Subtracting time passed until next arrival from all server workloads
            server_workloads = max(server_workloads, 0)  # All those with workload < 0 are converted to 0
        
        i += 1

##################################################################################################################################
        
    # Post processing
    
#     effective_arrival_times = np.array(effective_arrival_times)
#     effective_interarrival_times = [effective_arrival_times[0]] + [j-i for i,j in zip(effective_arrival_times, effective_arrival_times[1:])]
#     effective_interarrival_times = np.array(effective_interarrival_times)
#     departure_times.sort()
#     departure_times = np.array(departure_times)
#     waiting_times = np.array(waiting_times)
#     upward_jump_virtual_waiting_times = np.array(upward_jump_virtual_waiting_times)
    
#     all_waiting_times = np.array(all_waiting_times)

    average_revenue = total_revenue/arrival_times[-1]
    
    return average_revenue

In [ ]:
def evaluate_simulated_theoretical_revenue(p):
    price = p
    
    N_runs = 100
    
    global simulated_revenue
    simulated_revenue = []

    for iter in range(N_runs):
        generate_arrival_process(lam)
        generate_service_times(service_par)

        avg_collections = queue_simulation_2(price)

        simulated_revenue.append(avg_collections)
        
        # print(iter)
        
    return mean(simulated_revenue)

## Studying the map $w \mapsto Q^p(w,a)$ for fixed price $p$ and action $a$.

In [ ]:
lam = 0.8
theta_1 = 0.1
theta_2 = 0.2
s = 1

service_par = 1

price = 10

In [ ]:
# Evaluating R(p) for using simulation

n = 5000000
theoretical_revenue = evaluate_simulated_theoretical_revenue(price)

print("Simulated value for R(p) = ", theoretical_revenue)

In [ ]:
# Evaluation of simulated Q values

n = 200000 # Number of customers per simulation run
N_runs = 500 # Number of simulations

initial_workload_vector = np.arange(0, 31, 1)

Q_values_joining_fixed_price = []
Q_values_not_joining_fixed_price = []

for initial_workload in initial_workload_vector:
    mean_Q_joining = 0
    mean_Q_not_joining = 0
    
    for iter in range(N_runs):
        generate_arrival_process(lam)
        generate_service_times(service_par)
        
        mean_Q_joining += queue_simulation_1(initial_workload, 1, price, theoretical_revenue)
        mean_Q_not_joining += queue_simulation_1(initial_workload, 0, price, theoretical_revenue)
    
    mean_Q_joining = mean_Q_joining/N_runs
    mean_Q_not_joining = mean_Q_not_joining/N_runs
    
    print(initial_workload)
    
    Q_values_joining_fixed_price.append(mean_Q_joining)
    Q_values_not_joining_fixed_price.append(mean_Q_not_joining)

In [ ]:
# Plotting simulated Q(w,a) versus s

plt.plot(initial_workload_vector, Q_values_joining_fixed_price, color="red", label = "Q values given w, a=1")
plt.plot(initial_workload_vector, Q_values_not_joining_fixed_price, color="blue", label = "Q values given w, a=0")
plt.xlabel("Intial workload at t=0")
plt.ylabel("Q value")
plt.legend()
plt.savefig("Q values - fixed price.pdf")

In [ ]:
## Plotting function approximation for Q(w,a) versus w

initial_workload_vector = np.arange(0, 20, 0.01)

approx_Q_values_joining_fixed_price = []
approx_Q_values_not_joining_fixed_price = []

for initial_workload in initial_workload_vector:
    temp_not_joining = -1/(1-joining_probability(initial_workload, price)) * gradient_joining_probability(initial_workload, price)
    temp_joining = gradient_joining_probability(initial_workload, price)/joining_probability(initial_workload, price)
    
    approx_Q_values_not_joining_fixed_price.append(temp_not_joining)
    approx_Q_values_joining_fixed_price.append(temp_joining)

plt.plot(initial_workload_vector, approx_Q_values_joining_fixed_price, color = "red", label = "Approx for Q values given w, a=1")
plt.plot(initial_workload_vector, approx_Q_values_not_joining_fixed_price, color = "blue", label = "Approx for Q values given w, a=0")
plt.xlabel("Intial workload at t=0")
plt.ylabel("Function approx for Q")
plt.legend()
plt.savefig("Approx Q values - fixed price.pdf")

## Studying the map $p \mapsto Q^p(w,a)$ for fixed initial workload $w$ and action $a$.

In [ ]:
lam = 0.8
theta_1 = 0.1
theta_2 = 0.2
s = 1

service_par = 1

initial_workload = 0

In [ ]:
# Evaluating R(p) for various prices using simulation

n = 1000000

prices = np.arange(1, 21, 1)
simulated_revenue_given_price = []

for price in prices:
    avg_revenue = evaluate_simulated_theoretical_revenue(price)
    simulated_revenue_given_price.append(avg_revenue)
    print(price)

tuples = [(key, value) for i, (key, value) in enumerate(zip(prices, simulated_revenue_given_price))]

price_revenue_dict = dict(tuples)

In [ ]:
plt.plot(prices, simulated_revenue_given_price)
plt.xlabel("Price p")
plt.ylabel("Simulated value for R(p)")
plt.title("R(p) vs p")
plt.savefig("Revenue versus price.pdf")

In [ ]:
n = 200000
N_runs = 500

Q_values_joining_fixed_initial_workload = []
Q_values_not_joining_fixed_initial_workload = []

for price in prices:
    mean_Q_joining = 0
    mean_Q_not_joining = 0
    
    for iter in range(N_runs):
        generate_arrival_process(lam)
        generate_service_times(1)
        
        mean_Q_joining += queue_simulation_1(initial_workload, 1, price, price_revenue_dict[price])
        mean_Q_not_joining += queue_simulation_1(initial_workload, 0, price, price_revenue_dict[price])
    
    mean_Q_joining = mean_Q_joining/N_runs
    mean_Q_not_joining = mean_Q_not_joining/N_runs
    
    print(price)
    
    Q_values_joining_fixed_initial_workload.append(mean_Q_joining)
    Q_values_not_joining_fixed_initial_workload.append(mean_Q_not_joining) 

In [ ]:
# Plotting simulated Q(w,a) versus w

plt.plot(prices, Q_values_joining_fixed_initial_workload, color="red", label = "Q values given price, a=1")
plt.plot(prices, Q_values_not_joining_fixed_initial_workload, color="blue", label = "Q values given price, a=0")
plt.xlabel("Price")
plt.ylabel("Q value")
plt.legend()
plt.savefig("Q values - fixed initial workload.pdf")

In [ ]:
## Plotting function approximation for Q(w,a) versus w

prices = np.arange(0, 20, 0.01)

approx_Q_values_joining_fixed_initial_workload = []
approx_Q_values_not_joining_fixed_initial_workload = []

for price in prices:
    temp_not_joining = -1/(1-joining_probability(initial_workload, price)) * gradient_joining_probability(initial_workload, price)
    temp_joining = gradient_joining_probability(initial_workload, price)/joining_probability(initial_workload, price)
    
    approx_Q_values_not_joining_fixed_initial_workload.append(temp_not_joining)
    approx_Q_values_joining_fixed_initial_workload.append(temp_joining)

plt.plot(prices, approx_Q_values_joining_fixed_initial_workload, color = "red", label = "Approx for Q values given price, a=1")
plt.plot(prices, approx_Q_values_not_joining_fixed_initial_workload, color = "blue", label = "Approx for Q values given price, a=0")
plt.xlabel("Price")
plt.ylabel("Function approx for Q")
plt.legend()
plt.savefig("Approx Q values - fixed initial workload.pdf")

## Verifying condition (3), Page 1060 from paper by Sutton

In [ ]:
# lam = 0.8
# theta_1 = 0.1
# theta_2 = 0.2
# s = 1
# service_par = 1

# price = 10

# gamma_par = 0.2633643762504345

# n = 200000 # Number of customers per simulation run
# N_runs = 500 # Number of simulations

# Q_values_random_sampling_0 = []
# Q_values_random_sampling_1 = []


# for initial_workload in sampled_states:
#     mean_Q_joining = 0
#     mean_Q_not_joining = 0
    
#     for iter in range(N_runs):
#         generate_arrival_process(lam)
#         generate_service_times(service_par)
        
#         mean_Q_joining += queue_simulation_1(initial_workload, 1, price, theoretical_revenue)
#         mean_Q_not_joining += queue_simulation_1(initial_workload, 0, price, theoretical_revenue)
    
#     mean_Q_joining = mean_Q_joining/N_runs
#     mean_Q_not_joining = mean_Q_not_joining/N_runs
    
#     print(i
    
#     Q_values_random_sampling_1.append(mean_Q_joining)
#     Q_values_random_sampling_0.append(mean_Q_not_joining)

          
# lhs = 0          

# for iter in range(len(sampled_states)):
#     lhs += joining_probability(sampled_states[iter], price) * (Q_values_random_sampling_1[iter] - approx_Q(sampled_states[iter], 1, price, gamma_par))
#           * gradient_approx_Q(sampled_states[iter], 1, price)
#     lhs += (1-joining_probability(sampled_states[iter], price)) * (Q_values_random_sampling_0[iter] - approx_Q(sampled_states[iter], 1, price, gamma_par))
#           * gradient_approx_Q(sampled_states[iter], 0, price)